# Turing patterns and your own automata (16 points)

Animal skin patterns are a beautiful and intriguing example of pattern formation in biology. They were studied by famous English mathematician Alan Turing, who, in 1952, published a paper titled *The Chemical Basis of Morphogenesis*. In his model, Turing described interactions between two homogeneneously distributed substances, that produce stable patterns. His model used partial differential equations. During this classes you'll implement a simpler, CA version of this model developed by [David Young](https://www.sciencedirect.com/science/article/abs/pii/0025556484900609).
![image](https://upload.wikimedia.org/wikipedia/commons/a/a1/TuringPattern.PNG)
<center>(source: <a href="https://upload.wikimedia.org/wikipedia/commons/a/a1/TuringPattern.PNG">wikimedia.org</a>)</center>

## David Young model

##### Grid
Two-dimensional rectangular grid.

##### Cells
Binary cells: `0` (inactive/passive) or `1` (active).

##### Neighbourhood
This model uses two Moore neighbourhoods. The first, with radius $R_a$, is used for determining the number of active cells (*short-range activation*). The second one, with greater radius $R_i$ is used for counting the amount of inactive/passive cells (*long-range inhibition*). $R_a$ and $R_i$ ($R_a<R_i$), as well as $w_a$ and $w_i$, are the model's parameters. We're considering only odd $R_a$ and $R_i$ values.

##### Model
Mathematically speaking, the model can be described as (Hiroki Sayama, *Introduction to the Modeling and Analysis of Complex Systems*):
* <center>Short-range neighbourhood:</center>
$$N_a = \left\{ x'\left| |x'|\right. \leq R_a \right\}$$
* <center>Long-range neighbourhood:</center>
 $$N_i = \left\{ x'\left| |x'|\right. \leq R_i \right\}$$
* <center>Transition function:</center>
 $$a_t(x) = w_a\sum_{x' \in N_a}s_t(x+x')-w_i\sum_{x' \in N_i}s_t(x+x')$$
* $s_t$ is a mathematical function that maps location to state, the sum is simply a sum of neighbours values
  $$s_{t+1}(x) = \left\{ \begin{array}{ll}1 & \text{if } a_t(x) > 0\\0 & \text{otherwise}\end{array}\right.$$


## Python implementation of David Young's model (4 points)
Fill in the gaps in the following code.

In [1]:
import numpy as np
import itertools
import pygame

from pygame.locals import (
    K_UP,
    K_DOWN,
    K_LEFT,
    K_RIGHT,
    K_ESCAPE,
    KEYDOWN,
    QUIT,
)

pygame 2.6.1 (SDL 2.28.4, Python 3.13.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Grid size
rows = 80
cols = 80

In [7]:
class Model:
    def __init__(self, rows, cols, Ra, Ri, wa, wi,p1):
        self.grid = np.random.choice([0, 1], size=(rows, cols), p=[1-p1, p1])
        # self.grid = np.zeros((rows, cols))
        # self.grid[rows//2-1:rows//2+1, cols//2-1:cols//2+1] = 1
        self.Ra = Ra
        self.Ri = Ri
        self.wa = wa
        self.wi = wi
    
    def update(self):
        rows, cols = self.grid.shape
        new_grid = self.grid.copy()
        for x in range(0, rows):
            for y in range(0, cols):
                a = np.sum(self.grid[x-self.Ra:x+self.Ra+1, y-self.Ra:y+self.Ra+1]) * self.wa - \
                    np.sum(self.grid[x-self.Ri:x+self.Ri+1, y-self.Ri:y+self.Ri+1]) * self.wi
                if a > 0:
                    new_grid[x][y] = 1
                elif a < 0:
                    new_grid[x][y] = 0
        self.grid = new_grid
        return

    def drawGrid(self, screen,w_width, w_height):
        black = (0,0,0)
        white = (255,255,255)
        rows, cols = self.grid.shape
        blockSize = (min(w_width, w_height)-max(rows, cols))/max(rows, cols)
        for x in range(self.Ri, rows-self.Ri):
            for y in range(self.Ri, cols-self.Ri):
                pos_x = (blockSize+1)*x
                pos_y = (blockSize+1)*y
                rect = pygame.Rect(pos_x, pos_y, blockSize, blockSize)
                if self.grid[x][y] == 1:
                    pygame.draw.rect(screen, black, rect, 0)
                else:
                    pygame.draw.rect(screen, white, rect, 0)    
        pygame.display.flip()

In [16]:
pygame.init()
w_width = 800
w_height = 800
# Set up the drawing window, adjust the size
screen = pygame.display.set_mode([w_width, w_height])
model = Model(rows,cols, 3, 5, 0.8, 0.4, 0.1)

# Set background
screen.fill((128, 128, 128))

model.drawGrid(screen, w_width, w_height)
    
running = True

time_delay = 100 # 0.1 s
timer_event = pygame.USEREVENT + 1
pygame.time.set_timer(timer_event, time_delay )

while running:
    for event in pygame.event.get():   
        if event.type == QUIT:
            running = False
        
        if event.type == KEYDOWN:
            if event.key == K_RIGHT:
                model.update()
                model.drawGrid(screen, w_width, w_height)
        # if event.type == timer_event:
        #         model.update()
        #         model.drawGrid(screen, w_width, w_height)


pygame.quit()

KeyboardInterrupt: 

## Analysis (5 points)
Check different values of $w_a, w_i, R_a, R_b$. Find 5 visually different stable or oscilating patterns. Write down the parameteres and present results. Two sets of parameteres are considered to be different if at least two of the parameters are **substantially** different.

## Improvements (3 points)
In order to get this points you have to complete all of the previous tasks. Add widgets for parameters changing (e.g. slider or text field)


# Your cellular automata (4 points)

In order to get this points you have to complete all of the previous tasks.
Find another example of CA based model, descirbe the rules and implement the automata